In [1]:
import os
%pwd

'd:\\Documents\\projects\\End-to-End-SETI-Classification-using-CNN-MLFlow-DVC\\research'

In [2]:
os.chdir("../")
%pwd

'd:\\Documents\\projects\\End-to-End-SETI-Classification-using-CNN-MLFlow-DVC'

In [3]:
from dataclasses import dataclass
from pathlib import Path

@dataclass(frozen=True)
class ModelTrainingConfig:
    root_dir: Path
    trained_model_path: Path
    updated_base_model_path: Path
    training_data_path: Path
    params_epochs: int
    params_batch_size: int
    params_is_augmentation: bool
    params_image_size: list

In [4]:
from cnnClassifier.constants import CONFIG_FILE_PATH, PARAMS_FILE_PATH
from cnnClassifier.utils.common import read_yaml, create_directories
import tensorflow as tf

In [5]:
class ConfigurationManager:
    def __init__(self, config_filepath=CONFIG_FILE_PATH, params_filepath=PARAMS_FILE_PATH) -> None:
        self.config = read_yaml(config_filepath)
        self.params = read_yaml(params_filepath)
        
        create_directories([self.config.artifacts_root])
    
    def get_model_training_config(self) -> ModelTrainingConfig:
        model_training = self.config.model_training
        prepare_base_model = self.config.prepare_base_model
        params = self.params
        training_data = os.path.join(self.config.data_ingestion.data_dir, "seti-sub\\seti_signals_sub")
        
        create_directories([model_training.root_dir])
        
        model_training_config = ModelTrainingConfig(
            root_dir=Path(model_training.root_dir),
            trained_model_path=Path(model_training.trained_model_path),
            updated_base_model_path=Path(prepare_base_model.updated_base_model_path),
            training_data_path=Path(training_data),
            params_epochs=params.EPOCHS,
            params_batch_size=params.BATCH_SIZE,
            params_image_size=params.IMAGE_SIZE,
            params_is_augmentation=params.AUGMENTATION
        )
        
        return model_training_config

In [6]:
class ModelTraining:
    def __init__(self, config: ModelTrainingConfig) -> None:
        self.config = config
    
    def get_base_model(self):
        self.model = tf.keras.models.load_model(
            self.config.updated_base_model_path
        )
    
    @staticmethod
    def save_model(path: Path, model: tf.keras.Model):
        model.save(path)

    
    def train(self):
        train_data = os.path.join(self.config.training_data_path, "train")
        val_data = os.path.join(self.config.training_data_path, "valid")
        
        train_dataset = tf.keras.preprocessing.image_dataset_from_directory(
                                train_data,
                                shuffle=True,
                                image_size = (self.config.params_image_size[0], self.config.params_image_size[0]),
                                batch_size = self.config.params_batch_size
                            )
        
        val_dataset = tf.keras.preprocessing.image_dataset_from_directory(
                                val_data,
                                shuffle=True,
                                image_size = (self.config.params_image_size[0], self.config.params_image_size[0]),
                                batch_size = self.config.params_batch_size
                            )
        
        train = train_dataset.cache().shuffle(1000).prefetch(buffer_size=tf.data.AUTOTUNE)
        val = val_dataset.cache().shuffle(100).prefetch(buffer_size=tf.data.AUTOTUNE)
        
        early_stopping = tf.keras.callbacks.EarlyStopping(monitor='val_accuracy', patience=10, restore_best_weights=True)

        self.model.fit(
            train,
            epochs=self.config.params_epochs,
            batch_size=self.config.params_batch_size,
            verbose=1,
            validation_data=val,
            callbacks=[early_stopping]
        )

        self.save_model(
            path=self.config.trained_model_path,
            model=self.model
        )

In [7]:
try:
    config = ConfigurationManager()
    model_training_config = config.get_model_training_config()
    model_training = ModelTraining(config=model_training_config)
    model_training.get_base_model()
    model_training.train()
except Exception as e:
    raise e

[2024-07-03 20:06:44,246: INFO: common: yaml file config\config.yaml loadded successfully]
[2024-07-03 20:06:44,249: INFO: common: yaml file params.yaml loadded successfully]
[2024-07-03 20:06:44,250: INFO: common: Created directory at: ['artifacts']]
[2024-07-03 20:06:44,251: INFO: common: Created directory at: ['artifacts/model_training']]
Found 350 files belonging to 7 classes.


d:\Documents\projects\End-to-End-SETI-Classification-using-CNN-MLFlow-DVC\venv\Lib\site-packages\keras\src\saving\saving_lib.py:576: UserWarning: Skipping variable loading for optimizer 'adam', because it has 394 variables whereas the saved optimizer has 2 variables. 
  saveable.load_own_variables(weights_store.get(inner_path))


Found 175 files belonging to 7 classes.
22/22 ━━━━━━━━━━━━━━━━━━━━ 115s 3s/step - accuracy: 0.1066 - loss: 5.3012 - val_accuracy: 0.1429 - val_loss: 313579776.0000
